In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm 
import random, time, os
from sklearn.model_selection import train_test_split
import pandas as pd
from kaggle_datasets import KaggleDatasets

tf.__version__

In [ ]:
# Loading image
def load_img(pth, lab):
    img = tf.io.read_file(pth)
    img = tf.image.decode_png(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img.set_shape((299, 299, 3))
    # img = tf.expand_dims(img, axis=0)
    # Convert lab to uint8
    lab = tf.cast(lab, tf.uint8)
    # lab.set_shape(1)
    return img, lab

In [ ]:
def one_hot(X, y):
    global NUM_CLASS
    y = tf.one_hot(y, NUM_CLASS)

    return X, y

In [ ]:
# CutMix Augmentation
def CutMix(X, y_):
    global CUTMIX_RATE, BATCH_SIZE
    
    _, W, H, _ = X.shape.as_list()
    imgs, labs = [], []

    for i in range(BATCH_SIZE):
        k = np.random.randint(0, BATCH_SIZE)
        
        # Contructing the cutmix image
        # =============================
        # Img ith and kth
        img_i = X[i,:,:,:]
        img_k = X[k,:,:,:]

        # Constructing bounding box for the mixed patch
        w, h = int((1-CUTMIX_RATE)**.5 * W), int((1-CUTMIX_RATE)**.5 * H)
        x, y = np.random.randint(0, W - w), np.random.randint(0, H - h)

        # Mask 
        M = np.ones(img_i.shape)
        M[x:x+w, y:y+h,:] = 0    
        M_neg = 1 - M

        M = tf.constant(M, dtype=tf.float32)
        M_neg = tf.constant(M_neg, dtype=tf.float32)

        img = tf.math.add(tf.math.multiply(M, img_i), tf.math.multiply(M_neg, img_k))
        imgs.append(img)
        
        # Contructing the label
        # ============================================
        y_i = y_[i,:]
        y_k = y_[k,:]

        lab = CUTMIX_RATE * y_i + (1 - CUTMIX_RATE) * y_k
        labs.append(lab)

    imgs = tf.reshape(tf.stack(imgs), (BATCH_SIZE, W, H, 3))
    labs = tf.reshape(tf.stack(labs), (BATCH_SIZE, NUM_CLASS))
    return imgs, labs

In [ ]:
def other_aug(X, y):
    global MAX_DELTA_BRIGHT, LOWER_CONTRAST, UPPER_CONTRAST
    X = tf.image.random_flip_left_right(X)
    X = tf.image.random_brightness(X, max_delta=MAX_DELTA_BRIGHT)
    X = tf.image.random_contrast(X, lower=LOWER_CONTRAST, upper=UPPER_CONTRAST)

    return X, y

In [ ]:
def Augment_data(X, y):
    X, y = CutMix(X, y)
    X, y = other_aug(X, y)

    return X, y

In [ ]:
def get_dataset(valid_size = .1):
    pad = lambda x: f"0{x}" if x < 10 else f"{x}"
    
    train = pd.read_csv(f"{PTH}/train.csv")
    train_pth = train.apply(lambda x: f"{GCS_PTH}/resized/train/{pad(x['category'])}/{x['filename']}", axis=1)
    train_lab = train["category"]

    test = pd.read_csv(f"{PTH}/test.csv")
    test_pth = test.apply(lambda x: f"{GCS_PTH}/resized/test/{x['filename']}", axis=1)
    test_lab = test["category"].values

    # Validation split:
    train_pth , valid_pth, train_lab, valid_lab = train_test_split(train_pth, train_lab, test_size = valid_size)  
    
    # Returning TF's Dataset API
    train = tf.data.Dataset.from_tensor_slices((train_pth, train_lab))
    valid = tf.data.Dataset.from_tensor_slices((valid_pth, valid_lab))
    test = tf.data.Dataset.from_tensor_slices((test_pth, test_lab))
    
    train = train.map(load_img, num_parallel_calls=AUTO)\
        .map(one_hot, num_parallel_calls=AUTO)\
        .batch(BATCH_SIZE)\
        .map(Augment_data, num_parallel_calls=AUTO)\
        .cache()
    
    valid = valid.map(load_img, num_parallel_calls=AUTO)\
        .map(one_hot, num_parallel_calls=AUTO)
        # .batch(BATCH_SIZE)
    
    test = test.map(load_img, num_parallel_calls=AUTO)\
        .map(one_hot, num_parallel_calls=AUTO)\
        .batch(BATCH_SIZE)
    
    return train, valid, test

In [ ]:
# Config TPU
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

In [ ]:
#  Efficient net preprocess input
class Preprocess(tf.keras.layers.Layer):
    def __init__(self):
        super(Preprocess, self).__init__()
    
    def call(self, X):
        X = tf.keras.applications.efficientnet.preprocess_input(X)
        return X

In [ ]:
def get_model():
    with strategy.scope():
        # Load efficient net
        Preprocess(),

        base = tf.keras.applications.efficientnet.EfficientNetB7(
            include_top = False,
            weights="imagenet",
            pooling = None
        )
        
        base.trainable=False

        net = tf.keras.models.Sequential([
            base,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(42, activation="softmax")
        ])

        net.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate=.1),
            loss = "categorical_crossentropy",
            metrics = "accuracy"
        )

        return net

In [ ]:
# Callbacks
# Reduce learning rate on plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=2, verbose=1,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=1e-5
)

# save checkspoint
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/gdrive/MyDrive/dataset/checkpoint_tpu/efficient_netB7",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# configs
PTH = "/kaggle/input/shopee-code-league-2020-product-detection"
NUM_CLASS = 42
CUTMIX_RATE = 0.66
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
AUTO = tf.data.experimental.AUTOTUNE
MAX_DELTA_BRIGHT = .2
LOWER_CONTRAST = .8
UPPER_CONTRAST = 1.2
GCS_PTH = KaggleDatasets().get_gcs_path()

In [ ]:
net = get_model()
train, valid, test = get_dataset()

history = net.fit(
    train, 
    validation_data = valid,
    callbacks=[reduce_lr],
    epochs=5
)